In [1]:
%load_ext autoreload
%autoreload 1

#  https://colab.research.google.com/notebooks/data_table.ipynb
from google.colab import data_table
data_table.enable_dataframe_formatter()
from google.colab import drive
drive.mount('/content/gdrive')

# https://stackoverflow.com/questions/52733786/how-to-import-custom-modules-in-google-colab
import sys
sys.path.append('/content/gdrive/MyDrive/SEMRUSH/')

%aimport my

import sys
import numpy as np
import pandas as pd
import os
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import pickle

import my
from my import p

pd.set_option("max_colwidth", 35)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from IPython.display import HTML, display

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
     
 # номер или название эксперимента
# n_exp = '1/'
dir_out = "/content/gdrive/MyDrive/SEMRUSH/out/"
dir_data = '/content/gdrive/MyDrive/SEMRUSH/data/'
os.makedirs(dir_out, exist_ok = True)
RANDOM_STATE = 34

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
t = pd.read_parquet(dir_data+'train_data.p')
p(t.shape)
t[:3]

(13040643, 27) 
 ~


,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,root_domain,...,ff_count_slash_target,ff_count_subdomain_target,cat_domain,cat_target,pair_target,pair_root_target,pair_ltd_target,pair_cat_target,pair_url_target,pair_path_url_target
0,0,224312,1177617984,189763,531346,0,0,1,2,157605,...,2,2,65,65,6218560189763,5176440157605,2310000233,650000065,16591760531346,15217731317010
1,0,224312,1177618316,804359,2108910,1,0,1,2,670068,...,2,2,65,65,6218560804359,5176440670068,2310000231,650000065,16591762108910,15217731531109
2,0,224312,1177618348,988665,2549286,2,0,1,2,823669,...,2,2,65,65,6218560988665,5176440823669,2310000231,650000065,16591762549286,15217731317010


In [3]:
FOLD = 2
folds = my.load_pickle(dir_data+'folds_3to3.pickle')
sts = my.load_pickle(dir_data+f'sts_split_3_fold_{FOLD}.pickle')
train_ids, val_ids = folds[FOLD]['level_2'][0][0],\
                                 folds[FOLD]['level_2'][0][1]
train_ids.shape, val_ids.shape

((44443,), (22226,))

In [4]:
t = t[t.device_id.isin(np.r_[train_ids,val_ids])]\
          .reset_index(drop=True).copy()
gc.collect()
t.shape, t.index

((4382444, 27), RangeIndex(start=0, stop=4382444, step=1))

In [5]:
t = my.add_stats_features(t, sts)
del sts
p(t.isna().sum().sum())
t[:3]

0 
 ~


,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,root_domain,...,ff_count_ltd_ref_ltd_domain,ff_count_cat_ref_cat_domain,ff_count_url_ref_url,ff_count_path_url_ref_path_url,ff_prob_ref,ff_prob_ref_root,ff_prob_ref_ltd,ff_prob_ref_cat,ff_prob_ref_url,ff_prob_ref_path_url
0,6,105686,1177148841,734188,5664542,0,0,3,1,611493,...,170.000,107.000,0.000,0.000,0.000,0.000,0.698,0.132,0.000,0.000
1,6,105686,1177148866,364000,997845,1,0,1,2,302560,...,260.000,110.000,38570.000,968659.000,0.000,0.000,0.932,0.157,0.000,0.085
2,6,105686,1177148872,195704,3717961,2,0,2,1,162503,...,170.000,110.000,92.000,968659.000,0.000,0.000,0.698,0.157,0.000,0.085


In [6]:
del_cols = ['root_domain', 'ltd_domain', 'root_target', 'ltd_target', 'path_url',
       'path_url_target', 'cat_domain', 'cat_target',
       'pair_root_target', 'pair_ltd_target', 'pair_cat_target',
       'pair_url_target', 'pair_path_url_target']
t.drop(columns=del_cols, inplace=True)

In [7]:
t = my.time_features(t)

In [8]:
t = t.sort_values(['event_group_id','pos']).reset_index(drop=True)

In [9]:
t = my.merge_stat_pair_counts(t,col='domain',\
                                merge_col='pair_target')
t = t.loc[t.pos != 10].reset_index(drop=True)
gc.collect()
t.shape, t.isna().sum().sum()

pair_domain_tar


((3984040, 56), 0)

In [11]:
%%capture
!pip install catboost

In [10]:
fs = my.select_cols(t,['ff'])
X_train = t[fs]
y_train = t.y

In [11]:
from catboost import CatBoostClassifier
for i in range(3):
    print('pos = ',i)
    
    cb = CatBoostClassifier(iterations=10000,
                            random_seed=RANDOM_STATE+i,
                            early_stopping_rounds=100,
                            auto_class_weights=None,
                            eval_metric=None,
                            task_type="GPU",
                            learning_rate=0.1)

    cb.fit(X_train, y_train, verbose=1000)
    
    cb.save_model('/content/gdrive/MyDrive/SEMRUSH/models/'+f'cb_fold_{FOLD}_{i}.cbm')

pos =  0
0:	learn: 0.5374776	total: 81.7ms	remaining: 13m 37s
1000:	learn: 0.1254089	total: 49.8s	remaining: 7m 27s
2000:	learn: 0.1216278	total: 1m 38s	remaining: 6m 33s
3000:	learn: 0.1192046	total: 2m 27s	remaining: 5m 44s
4000:	learn: 0.1172450	total: 3m 17s	remaining: 4m 55s
5000:	learn: 0.1155974	total: 4m 6s	remaining: 4m 6s
6000:	learn: 0.1141180	total: 4m 57s	remaining: 3m 18s
7000:	learn: 0.1128241	total: 5m 46s	remaining: 2m 28s
8000:	learn: 0.1116409	total: 6m 36s	remaining: 1m 39s
9000:	learn: 0.1105453	total: 7m 25s	remaining: 49.4s
9999:	learn: 0.1095214	total: 8m 15s	remaining: 0us
pos =  1
0:	learn: 0.5379140	total: 80ms	remaining: 13m 19s
1000:	learn: 0.1255285	total: 50.5s	remaining: 7m 33s
2000:	learn: 0.1216846	total: 1m 40s	remaining: 6m 39s
3000:	learn: 0.1191969	total: 2m 29s	remaining: 5m 48s
4000:	learn: 0.1172499	total: 3m 19s	remaining: 4m 59s
5000:	learn: 0.1156290	total: 4m 9s	remaining: 4m 9s
6000:	learn: 0.1142215	total: 4m 58s	remaining: 3m 19s
7000:	le

In [8]:
train = t[t.device_id.isin(train_ids)]\
     .sort_values(['event_group_id','pos']).reset_index(drop=True)
train.to_parquet(dir_data+'train.p',index=False)

val = t[t.device_id.isin(val_ids)]\
    .sort_values(['event_group_id','pos']).reset_index(drop=True)
val.to_parquet(dir_data+'val.p',index=False)

del t
gc.collect()

print(train.isna().sum().sum(),val.isna().sum().sum(),np.isinf(train).sum().sum(),np.isinf(val).sum().sum())
train.shape, val.shape

0 0 0 0


((2894386, 46), (1444476, 46))

In [ ]:
train = pd.read_parquet(dir_data+'train.p')
val = pd.read_parquet(dir_data+'val.p')
train.shape, val.shape

In [9]:
train = my.merge_stat_pair_counts(train,col='domain',\
                                merge_col='pair_target')
train = train.loc[train.pos != 10].reset_index(drop=True)
gc.collect()
train.shape, train.isna().sum().sum()

pair_domain_tar


((2631260, 56), 0)

In [10]:
val = my.merge_stat_pair_counts(val,col='domain',\
                                merge_col='pair_target')
val = val.loc[val.pos != 10].reset_index(drop=True)
gc.collect()
val.shape, val.isna().sum().sum()

pair_domain_tar


((1313160, 56), 0)

In [16]:
from catboost import CatBoostClassifier
fs = my.select_cols(train,['ff'])

y_train = train.y
y_val = val.y

res = []

for i in range(len(fs)):
  fs1 = fs.copy()
  feat = fs1.pop(i)

  X_train = train[fs1]
  X_val = val[fs1]

  cb = CatBoostClassifier(iterations=1000,
                            random_seed=RANDOM_STATE,
                            early_stopping_rounds=100,
                            auto_class_weights=None,
                            eval_metric=None,
                            task_type="GPU",
                            learning_rate=0.1)

  cb.fit(X_train, y_train, eval_set=(X_val, y_val), sample_weight=None,verbose=500)
  res.append([feat,cb.get_best_score()])
res = pd.DataFrame(res)

0:	learn: 0.5367878	test: 0.5387336	best: 0.5387336 (0)	total: 56.5ms	remaining: 56.4s
500:	learn: 0.1293823	test: 0.1328878	best: 0.1328878 (500)	total: 17.2s	remaining: 17.1s
999:	learn: 0.1254858	test: 0.1304283	best: 0.1304282 (998)	total: 33.9s	remaining: 0us
bestTest = 0.1304282466
bestIteration = 998
Shrink model to first 999 iterations.
0:	learn: 0.5367878	test: 0.5387337	best: 0.5387337 (0)	total: 56.2ms	remaining: 56.2s
500:	learn: 0.1291853	test: 0.1327559	best: 0.1327559 (500)	total: 17.4s	remaining: 17.3s
999:	learn: 0.1253862	test: 0.1303692	best: 0.1303692 (999)	total: 34.5s	remaining: 0us
bestTest = 0.1303691929
bestIteration = 999
0:	learn: 0.5367878	test: 0.5387336	best: 0.5387336 (0)	total: 69.9ms	remaining: 1m 9s
500:	learn: 0.1291925	test: 0.1326720	best: 0.1326719 (499)	total: 17.8s	remaining: 17.7s
999:	learn: 0.1253799	test: 0.1303234	best: 0.1303234 (999)	total: 35.3s	remaining: 0us
bestTest = 0.1303234183
bestIteration = 999
0:	learn: 0.5367878	test: 0.5387336

In [17]:
res

,0,1
0,ff_count_slash,{'learn': {'Logloss': 0.1254857...
1,ff_count_subdomain,{'learn': {'Logloss': 0.1253862...
2,ff_count_slash_target,{'learn': {'Logloss': 0.1253798...
3,ff_count_subdomain_target,{'learn': {'Logloss': 0.1254279...
4,ff_pair_count_domain,{'learn': {'Logloss': 0.1253567...
5,ff_pair_count_root_domain,{'learn': {'Logloss': 0.1252350...
6,ff_pair_count_ltd_domain,{'learn': {'Logloss': 0.1252883...
7,ff_pair_count_cat_domain,{'learn': {'Logloss': 0.1253005...
8,ff_pair_count_url,{'learn': {'Logloss': 0.1251905...
9,ff_pair_count_path_url,{'learn': {'Logloss': 0.1252274...


In [ ]:
fs = my.select_cols(train,['ff'])

X_train = train[fs]
y_train = train.y

X_val = val[fs]
y_val = val.y

In [ ]:
from catboost import CatBoostClassifier

probs_val = val[['event_group_id','y']].copy()

cb = CatBoostClassifier(iterations=2000,
                            random_seed=RANDOM_STATE,
                            early_stopping_rounds=100,
                            auto_class_weights=None,
                            eval_metric=None,
                            task_type="GPU",
                            learning_rate=0.1)

cb.fit(X_train, y_train, eval_set=(X_val, y_val), sample_weight=None,verbose=500)
probs_val['probs'] = cb.predict_proba(X_val)[:,1]

0:	learn: 0.5367660	test: 0.5387149	best: 0.5387149 (0)	total: 56.4ms	remaining: 1m 52s
500:	learn: 0.1291108	test: 0.1325720	best: 0.1325720 (500)	total: 17.4s	remaining: 51.9s
1000:	learn: 0.1252058	test: 0.1301105	best: 0.1301105 (1000)	total: 34.4s	remaining: 34.3s
1500:	learn: 0.1227743	test: 0.1288998	best: 0.1288998 (1500)	total: 51.4s	remaining: 17.1s
1999:	learn: 0.1209351	test: 0.1281246	best: 0.1281244 (1998)	total: 1m 8s	remaining: 0us
bestTest = 0.1281243694
bestIteration = 1998
Shrink model to first 1999 iterations.


In [ ]:
_ = my.scores(probs_val)

f1: 0.703 | precision: 0.787 | recall: 0.642 |


,f1,precision,recall
0,0.637,0.776,0.541
1,0.644,0.770,0.553
2,0.649,0.783,0.554
3,0.665,0.787,0.576
4,0.668,0.779,0.585
5,0.682,0.787,0.601
6,0.710,0.788,0.647
7,0.728,0.789,0.675
8,0.766,0.795,0.740
9,0.879,0.818,0.950


In [ ]:
X_train.drop(columns=[ 'ff_delta_time',
 'ff_ratio_time',
 'ff_device_ratio_time'],inplace=True)

X_val.drop(columns=[ 'ff_delta_time',
 'ff_ratio_time',
 'ff_device_ratio_time'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
probs_val = val[['event_group_id','y']].copy()

cb = CatBoostClassifier(iterations=3000,
                            random_seed=RANDOM_STATE,
                            early_stopping_rounds=100,
                            auto_class_weights=None,
                            eval_metric=None,
                            task_type="GPU",
                            learning_rate=0.1)

cb.fit(X_train, y_train, eval_set=(X_val, y_val), sample_weight=None,verbose=500)
probs_val['probs'] = cb.predict_proba(X_val)[:,1]

0:	learn: 0.5395101	test: 0.5413533	best: 0.5413533 (0)	total: 55.5ms	remaining: 2m 46s
500:	learn: 0.1389598	test: 0.1413615	best: 0.1413615 (500)	total: 17s	remaining: 1m 25s
1000:	learn: 0.1353858	test: 0.1391538	best: 0.1391538 (1000)	total: 33.9s	remaining: 1m 7s
1500:	learn: 0.1332472	test: 0.1380907	best: 0.1380896 (1498)	total: 50.9s	remaining: 50.8s
2000:	learn: 0.1316298	test: 0.1374289	best: 0.1374289 (2000)	total: 1m 7s	remaining: 33.9s
2500:	learn: 0.1303069	test: 0.1369880	best: 0.1369873 (2496)	total: 1m 24s	remaining: 16.9s
2999:	learn: 0.1291805	test: 0.1366397	best: 0.1366397 (2999)	total: 1m 41s	remaining: 0us
bestTest = 0.1366396826
bestIteration = 2999


In [ ]:
_ = my.scores(probs_val)

f1: 0.672 | precision: 0.697 | recall: 0.651 |


,f1,precision,recall
0,0.594,0.597,0.590
1,0.597,0.606,0.589
2,0.607,0.634,0.582
3,0.623,0.653,0.596
4,0.638,0.678,0.603
5,0.654,0.705,0.609
6,0.685,0.722,0.651
7,0.709,0.757,0.667
8,0.752,0.794,0.715
9,0.864,0.827,0.904


In [ ]:


models = []
results = []
fs = my.select_cols(train,['ff'])

probs_val = val[['event_group_id','y']].copy()
probs_val['probs'] = 0

n=10

for i in range(n):
    print('pos = ',i)
    X_train = train.loc[train.pos==i,fs]
    y_train = train.loc[train.pos==i,'y']

    X_val = val.loc[val.pos==i,fs]
    y_val = val.loc[val.pos==i,'y']
    
    cb = CatBoostClassifier(iterations=2000,
                            random_seed=RANDOM_STATE,
                            early_stopping_rounds=100,
                            auto_class_weights=None,
                            eval_metric=None,
                            task_type="GPU",
                            learning_rate=0.1)

    cb.fit(X_train, y_train, eval_set=(X_val, y_val), sample_weight=None,verbose=500)
    
    models.append(cb)
    probs_val.loc[val.pos==i,'probs'] = cb.predict_proba(X_val)[:,1]

In [ ]:
results = my.pair_counts(train,'domain')
p(results.shape, results.isna().sum())
results[:3]

(21953620, 4) 
 ~
device_id          0
pair_domain_tar    0
diff_timestamp     0
diff_pos           0
dtype: int64 
 ~


,device_id,pair_domain_tar,diff_timestamp,diff_pos
1,131803,6058950574675,-1,-1
2,131803,6058950338769,-28,-2
3,131803,6058950574675,-30,-3


In [ ]:
df_stat = my.stat_pair_counts(results)
p(df_stat.shape)
df_stat[:3]

pair_domain_tar
(7185212, 10) 
 ~


,ffdiff_pos_count_0,ffdiff_pos_count_1,ffdiff_pos_mean_0,ffdiff_pos_mean_1,ffdiff_pos_sum_0,ffdiff_pos_sum_1,ffdiff_timestamp_mean_0,ffdiff_timestamp_mean_1,ffdiff_timestamp_sum_0,ffdiff_timestamp_sum_1
pair_domain_tar,,,,,,,,,,
6058950574675,0.353,0.647,-2.944,4.333,0.270,0.730,-539.056,359.636,0.450,0.550
6058950338769,1.000,0.000,-2.000,0.000,1.000,0.000,-28.000,0.000,1.000,0.000
6058950977222,0.500,0.500,-2.800,5.200,0.350,0.650,-90.600,603.400,0.131,0.869


In [ ]:
train = train.merge(df_stat,left_on='pair_target',right_index=True)
train.sort_values(['event_group_id','pos'],inplace=True)
train.reset_index(drop=True,inplace=True)
gc.collect()
p(train.shape)
train[:5]

(2844523, 56) 
 ~


,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,target,...,ffdiff_pos_count_0,ffdiff_pos_count_1,ffdiff_pos_mean_0,ffdiff_pos_mean_1,ffdiff_pos_sum_0,ffdiff_pos_sum_1,ffdiff_timestamp_mean_0,ffdiff_timestamp_mean_1,ffdiff_timestamp_sum_0,ffdiff_timestamp_sum_1
0,11,131803,1176564102,605895,6852550,0,0,2,2,908230,...,0.500,0.500,-4.000,10.000,0.286,0.714,-209.000,1011.000,0.171,0.829
1,11,131803,1176564103,574675,5169457,1,0,3,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747
3,11,131803,1176564132,574675,1512335,3,0,1,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747
4073,2553,214058,1175458958,574675,6150632,3,0,1,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747
118615,71611,258486,1178355398,574675,1513008,2,0,1,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747


In [ ]:
val = my.merge_stat_pair_counts(val,col='domain',\
                                merge_col='pair_target')
val.shape, val.isna().sum().sum()

pair_domain_tar


((1414247, 56), 0)

In [ ]:
train = train.loc[train.pos != 10].reset_index(drop=True)
val = val.loc[val.pos != 10].reset_index(drop=True)

In [ ]:
fs = my.select_cols(train,['ff'])
# https://catboost.ai/en/docs/references/training-parameters/common#auto_class_weights

X_train = train[fs]
y_train = train.y

X_val = val[fs]
y_val = val.y
X_train.shape

(2631260, 46)

In [ ]:
cb, config = my.fit_catboost(X_train,
                             y_train,
                             X_val,
                             y_val,
                             lr=0.1,
                             iters=3000,
                             random_state=RANDOM_STATE,
                             dir_out=dir_out)

0:	learn: 0.5370336	test: 0.5394919	best: 0.5394919 (0)	total: 968ms	remaining: 48m 24s
500:	learn: 0.1285649	test: 0.1321401	best: 0.1321401 (500)	total: 8m 29s	remaining: 42m 23s
1000:	learn: 0.1242120	test: 0.1294811	best: 0.1294802 (999)	total: 16m 48s	remaining: 33m 33s
1500:	learn: 0.1213720	test: 0.1280873	best: 0.1280873 (1500)	total: 25m 4s	remaining: 25m 2s
2000:	learn: 0.1191847	test: 0.1272736	best: 0.1272735 (1999)	total: 33m 18s	remaining: 16m 37s
2500:	learn: 0.1173520	test: 0.1266997	best: 0.1266997 (2500)	total: 41m 25s	remaining: 8m 15s
2999:	learn: 0.1157206	test: 0.1262511	best: 0.1262511 (2999)	total: 49m 41s	remaining: 0us

bestTest = 0.1262510936
bestIteration = 2999

model_LR_0.1_SEED_34.cbm


In [ ]:
f1, precision, recall = my.scores(val, cb.predict_proba(X_val)[:, 1])

p('train f1:', round(f1_score(y_train, cb.predict(X_train)),3))
p(f'f1: {f1} | precision: {precision} | recall: {recall} |')
my.scores_no_avg(val,cb.predict_proba(X_val)[:, 1])

train f1: 
 ~
0.767 
 ~
f1: 0.708 | precision: 0.788 | recall: 0.65 | 
 ~


,f1,precision,recall
0,0.643,0.767,0.554
1,0.652,0.776,0.563
2,0.655,0.783,0.563
3,0.673,0.787,0.588
4,0.676,0.785,0.594
5,0.688,0.789,0.610
6,0.715,0.788,0.654
7,0.729,0.787,0.678
8,0.770,0.798,0.744
9,0.882,0.823,0.951


In [ ]:
pd.set_option("max_colwidth", 35)
imp_feat = cb.get_feature_importance(prettified=True)
imp_feat

,Feature Id,Importances
0,ffdiff_pos_mean_1,9.260
1,ff_count_dom_ref,8.407
2,ff_pair_count_domain,6.307
3,ff_count_dev_ref,5.821
4,ff_count_dev_root_ref,5.627
5,ff_prob_ref,5.366
6,ff_ratio_time,4.715
7,ff_count_root_ref_root_domain,4.328
8,ff_count_dom_root_ref,3.629
9,ff_prob_ref_root,3.451


In [ ]:
results[:12]

,device_id,pair_domain_tar,diff_timestamp,diff_pos
1,131803,6058950574675,-1,-1
2,131803,6058950338769,-28,-2
3,131803,6058950574675,-30,-3
4,131803,6058950977222,-41,-4
5,131803,6058950631792,-110,-5
6,131803,6058951220641,-145,-6
7,131803,6058950364000,-307,-7
8,131803,6058950214463,-672,-8
9,131803,6058950908233,-971,-9
10,131803,6058950908230,-1011,-10


In [ ]:
results[results.pair_domain_tar==1745100174510]

,device_id,pair_domain_tar,diff_timestamp,diff_pos


In [ ]:
results[results.pair_domain_tar==11519911151991]

,device_id,pair_domain_tar,diff_timestamp,diff_pos
1123236,201962,11519911151991,-8,-4
115636,166574,11519911151991,-190,-3
115633,166574,11519911151991,190,3
1123232,201962,11519911151991,8,4


In [ ]:
train[(train.domain == 1151991)  & (train.device_id == 201962)]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,target,...,ff_count_path_url_ref_path_url,ff_prob_ref,ff_prob_ref_root,ff_prob_ref_ltd,ff_prob_ref_cat,ff_prob_ref_url,ff_prob_ref_path_url,ff_delta_time,ff_ratio_time,ff_device_ratio_time
1123232,678088,201962,1170192527,1151991,3560344,0,0,2,2,1171099,...,0.000,0.000,0.000,0.932,0.938,0.000,0.000,1228,1.000,18.712
1123236,678088,201962,1170192535,1151991,3560344,4,0,2,2,1171099,...,0.000,0.000,0.000,0.932,0.938,0.000,0.000,1220,0.993,18.590


In [ ]:
train[train.event_group_id==678088]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,target,...,ff_count_path_url_ref_path_url,ff_prob_ref,ff_prob_ref_root,ff_prob_ref_ltd,ff_prob_ref_cat,ff_prob_ref_url,ff_prob_ref_path_url,ff_delta_time,ff_ratio_time,ff_device_ratio_time
1123232,678088,201962,1170192527,1151991,3560344,0,0,2,2,1171099,...,0.000,0.000,0.000,0.932,0.938,0.000,0.000,1228,1.000,18.712
1123233,678088,201962,1170192533,817286,4706499,1,0,3,3,1171099,...,0.000,0.000,0.000,0.932,0.788,0.000,0.000,1222,0.995,18.620
1123234,678088,201962,1170192533,533462,1426983,2,0,7,2,1171099,...,11.000,0.000,0.000,0.837,0.759,0.000,0.000,1222,0.995,18.620
1123235,678088,201962,1170192534,237884,7031946,3,0,4,2,1171099,...,0.000,0.000,0.000,0.932,0.938,0.000,0.000,1221,0.994,18.605
1123236,678088,201962,1170192535,1151991,3560344,4,0,2,2,1171099,...,0.000,0.000,0.000,0.932,0.938,0.000,0.000,1220,0.993,18.590
1123237,678088,201962,1170192535,842109,6854350,5,0,2,2,1171099,...,0.000,0.000,0.000,0.932,0.938,0.000,0.000,1220,0.993,18.590
1123238,678088,201962,1170192536,574675,6508902,6,0,6,2,1171099,...,9.000,0.000,0.000,0.932,0.938,0.000,0.000,1219,0.993,18.575
1123239,678088,201962,1170193451,923972,6046788,7,0,2,2,1171099,...,969015.000,0.000,0.000,0.932,0.889,0.000,0.003,304,0.248,4.632
1123240,678088,201962,1170193639,430728,6046232,8,0,2,2,1171099,...,969015.000,0.000,0.000,0.932,0.889,0.000,0.003,116,0.094,1.768
1123241,678088,201962,1170193753,1008333,2587833,9,1,2,2,1171099,...,0.000,0.000,0.000,0.827,0.889,0.000,0.000,2,0.002,0.030


In [ ]:
train[:12]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,target,...,ff_count_path_url_ref_path_url,ff_prob_ref,ff_prob_ref_root,ff_prob_ref_ltd,ff_prob_ref_cat,ff_prob_ref_url,ff_prob_ref_path_url,ff_delta_time,ff_ratio_time,ff_device_ratio_time
0,11,131803,1176564102,605895,6852550,0,0,2,2,908230,...,0.000,0.000,0.000,0.000,0.661,0.000,0.000,1011,1.000,8.708
1,11,131803,1176564103,574675,5169457,1,0,3,2,908230,...,295.000,0.000,0.000,0.018,0.938,0.000,0.000,1010,0.999,8.699
2,11,131803,1176564130,338769,5258964,2,0,1,3,908230,...,13.000,0.000,0.000,0.006,0.788,0.000,0.000,983,0.972,8.467
3,11,131803,1176564132,574675,1512335,3,0,1,2,908230,...,59662.000,0.000,0.000,0.018,0.938,0.000,0.000,981,0.970,8.450
4,11,131803,1176564143,977222,2521369,4,0,4,2,908230,...,0.000,0.000,0.000,0.018,0.674,0.000,0.000,970,0.959,8.355
5,11,131803,1176564212,631792,1682624,5,0,1,2,908230,...,0.000,0.000,0.000,0.018,0.938,0.000,0.000,901,0.891,7.761
6,11,131803,1176564247,1220641,3077191,6,0,2,2,908230,...,0.000,0.000,0.000,0.009,0.938,0.000,0.000,866,0.857,7.459
7,11,131803,1176564409,364000,997920,7,0,1,2,908230,...,4937.000,0.000,0.000,0.018,0.938,0.000,0.000,704,0.696,6.064
8,11,131803,1176564774,214463,5224717,8,1,2,3,908230,...,969015.000,0.261,0.761,0.074,0.830,0.000,0.000,339,0.335,2.920
9,11,131803,1176565073,908233,6268352,9,0,2,1,908230,...,107.000,0.622,0.615,0.010,0.938,0.000,0.000,40,0.040,0.345


In [ ]:
q = my.stat_pair_counts(results)
p(q.shape)
q[:12]

pair_domain_tar
(7185212, 10) 
 ~


,ffdiff_pos_count_0,ffdiff_pos_count_1,ffdiff_pos_mean_0,ffdiff_pos_mean_1,ffdiff_pos_sum_0,ffdiff_pos_sum_1,ffdiff_timestamp_mean_0,ffdiff_timestamp_mean_1,ffdiff_timestamp_sum_0,ffdiff_timestamp_sum_1
pair_domain_tar,,,,,,,,,,
6058950574675,0.353,0.647,-2.944,4.333,0.270,0.730,-539.056,359.636,0.450,0.550
6058950338769,1.000,0.000,-2.000,0.000,1.000,0.000,-28.000,0.000,1.000,0.000
6058950977222,0.500,0.500,-2.800,5.200,0.350,0.650,-90.600,603.400,0.131,0.869
6058950631792,1.000,0.000,-4.333,0.000,1.000,0.000,-508.000,0.000,1.000,0.000
6058951220641,1.000,0.000,-7.000,0.000,1.000,0.000,-569.000,0.000,1.000,0.000
6058950364000,0.538,0.462,-3.571,3.667,0.532,0.468,-351.571,397.000,0.508,0.492
6058950214463,0.500,0.500,-8.000,6.000,0.571,0.429,-672.000,286.000,0.701,0.299
6058950908233,1.000,0.000,-6.000,0.000,1.000,0.000,-511.500,0.000,1.000,0.000
6058950908230,0.500,0.500,-10.000,4.000,0.714,0.286,-1011.000,209.000,0.829,0.171


In [ ]:
train.columns

Index(['event_group_id', 'device_id', 'timestamp', 'domain', 'url', 'pos', 'y',
       'ff_count_slash', 'ff_count_subdomain', 'target', 'url_target',
       'ff_count_slash_target', 'ff_count_subdomain_target', 'pair_target',
       'ff_pair_count_domain', 'ff_pair_count_root_domain',
       'ff_pair_count_ltd_domain', 'ff_pair_count_cat_domain',
       'ff_pair_count_url', 'ff_pair_count_path_url', 'ff_count_dev_ref',
       'ff_count_dom_ref', 'ff_count_dev_root_ref', 'ff_count_dom_root_ref',
       'ff_count_dev_ltd_ref', 'ff_count_dom_ltd_ref', 'ff_count_dev_cat_ref',
       'ff_count_dom_cat_ref', 'ff_count_dev_url_ref', 'ff_count_dom_url_ref',
       'ff_count_dev_path_url_ref', 'ff_count_dom_path_url_ref',
       'ff_count_root_ref_root_domain', 'ff_count_ltd_ref_ltd_domain',
       'ff_count_cat_ref_cat_domain', 'ff_count_url_ref_url',
       'ff_count_path_url_ref_path_url', 'ff_prob_ref', 'ff_prob_ref_root',
       'ff_prob_ref_ltd', 'ff_prob_ref_cat', 'ff_prob_ref_url',
   

In [ ]:
train = train.merge(q,left_on='pair_target',right_index=True)
gc.collect()
p(train.shape)
train[:5]

(2844523, 56) 
 ~


,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,target,...,ffdiff_pos_count_0,ffdiff_pos_count_1,ffdiff_pos_mean_0,ffdiff_pos_mean_1,ffdiff_pos_sum_0,ffdiff_pos_sum_1,ffdiff_timestamp_mean_0,ffdiff_timestamp_mean_1,ffdiff_timestamp_sum_0,ffdiff_timestamp_sum_1
0,11,131803,1176564102,605895,6852550,0,0,2,2,908230,...,0.500,0.500,-4.000,10.000,0.286,0.714,-209.000,1011.000,0.171,0.829
1,11,131803,1176564103,574675,5169457,1,0,3,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747
3,11,131803,1176564132,574675,1512335,3,0,1,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747
4073,2553,214058,1175458958,574675,6150632,3,0,1,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747
118615,71611,258486,1178355398,574675,1513008,2,0,1,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747


In [ ]:
train.sort_values(['event_group_id','pos'],inplace=True)
train.reset_index(drop=True,inplace=True)
train[:12]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,target,...,ffdiff_pos_count_0,ffdiff_pos_count_1,ffdiff_pos_mean_0,ffdiff_pos_mean_1,ffdiff_pos_sum_0,ffdiff_pos_sum_1,ffdiff_timestamp_mean_0,ffdiff_timestamp_mean_1,ffdiff_timestamp_sum_0,ffdiff_timestamp_sum_1
0,11,131803,1176564102,605895,6852550,0,0,2,2,908230,...,0.500,0.500,-4.000,10.000,0.286,0.714,-209.000,1011.000,0.171,0.829
1,11,131803,1176564103,574675,5169457,1,0,3,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747
2,11,131803,1176564130,338769,5258964,2,0,1,3,908230,...,0.000,1.000,0.000,8.000,0.000,1.000,0.000,983.000,0.000,1.000
3,11,131803,1176564132,574675,1512335,3,0,1,2,908230,...,0.261,0.739,-3.000,5.980,0.150,0.850,-446.944,466.725,0.253,0.747
4,11,131803,1176564143,977222,2521369,4,0,4,2,908230,...,0.286,0.714,-3.500,6.000,0.189,0.811,-136.500,581.200,0.086,0.914
5,11,131803,1176564212,631792,1682624,5,0,1,2,908230,...,0.000,1.000,0.000,7.000,0.000,1.000,0.000,1108.000,0.000,1.000
6,11,131803,1176564247,1220641,3077191,6,0,2,2,908230,...,0.000,1.000,0.000,4.000,0.000,1.000,0.000,866.000,0.000,1.000
7,11,131803,1176564409,364000,997920,7,0,1,2,908230,...,0.294,0.706,-4.000,5.833,0.222,0.778,-634.800,633.667,0.294,0.706
8,11,131803,1176564774,214463,5224717,8,1,2,3,908230,...,0.119,0.881,-3.200,2.595,0.143,0.857,-509.200,254.243,0.213,0.787
9,11,131803,1176565073,908233,6268352,9,0,2,1,908230,...,0.138,0.862,-2.750,2.680,0.141,0.859,-252.375,224.900,0.152,0.848


In [ ]:
q1 = train.groupby('event_group_id').size()
q1

event_group_id
11         11
18         11
27         11
31         11
34         11
           ..
1737982    11
1737991    11
1737997    10
1738004    10
1738007    11
Length: 263126, dtype: int64

In [ ]:
targets = set(train.loc[train.pos==10,'domain'])

In [ ]:
1151991 in targets, 174510 in targets

(True, True)

In [ ]:
train.loc[train['event_group_id']==1738007,['domain','target','pair_target','pos']]

,domain,target,pair_target,pos
2894375,179968,1151991,11519910179968,0
2894376,805906,1151991,11519910805906,1
2894377,574621,1151991,11519910574621,2
2894378,928208,1151991,11519910928208,3
2894379,237884,1151991,11519910237884,4
2894380,974684,1151991,11519910974684,5
2894381,355451,1151991,11519910355451,6
2894382,574621,1151991,11519910574621,7
2894383,838614,1151991,11519910838614,8
2894384,833394,1151991,11519910833394,9


In [ ]:
train.loc[train['event_group_id']==1737997,['domain','target','pair_target','pos']]

,domain,target,pair_target,pos
2894353,138684,174510,1745100138684,0
2894354,1180590,174510,1745101180590,1
2894355,504118,174510,1745100504118,2
2894356,773005,174510,1745100773005,3
2894357,88069,174510,1745100088069,4
2894358,574713,174510,1745100574713,5
2894359,829772,174510,1745100829772,6
2894360,504118,174510,1745100504118,7
2894361,360121,174510,1745100360121,8
2894362,834629,174510,1745100834629,9


In [ ]:
train.loc[train['event_group_id']==1738007,['domain','target','pair_target','pos']]

,domain,target,pair_target,pos
2844512,179968,1151991,11519910179968,0
2844513,805906,1151991,11519910805906,1
2844514,574621,1151991,11519910574621,2
2844515,928208,1151991,11519910928208,3
2844516,237884,1151991,11519910237884,4
2844517,974684,1151991,11519910974684,5
2844518,355451,1151991,11519910355451,6
2844519,574621,1151991,11519910574621,7
2844520,838614,1151991,11519910838614,8
2844521,833394,1151991,11519910833394,9


In [ ]:
q.columns

Index(['ffdiff_pos_count_0', 'ffdiff_pos_count_1', 'ffdiff_pos_mean_0',
       'ffdiff_pos_mean_1', 'ffdiff_pos_sum_0', 'ffdiff_pos_sum_1',
       'ffdiff_timestamp_mean_0', 'ffdiff_timestamp_mean_1',
       'ffdiff_timestamp_sum_0', 'ffdiff_timestamp_sum_1'],
      dtype='object')

In [ ]:
q.loc[11519911151991]

ffdiff_pos_count_0          0.500
ffdiff_pos_count_1          0.500
ffdiff_pos_mean_0          -3.500
ffdiff_pos_mean_1           3.500
ffdiff_pos_sum_0            0.500
ffdiff_pos_sum_1            0.500
ffdiff_timestamp_mean_0   -99.000
ffdiff_timestamp_mean_1    99.000
ffdiff_timestamp_sum_0      0.500
ffdiff_timestamp_sum_1      0.500
Name: 11519911151991, dtype: float32

In [ ]:
train.loc[train.pair_target == 11519911151991,['event_group_id','domain','target','pair_target','pos']]

,event_group_id,domain,target,pair_target,pos
910996,559390,1151991,1151991,11519911151991,10
1051352,645559,1151991,1151991,11519911151991,10
1448923,890785,1151991,1151991,11519911151991,10
1951015,1197961,1151991,1151991,11519911151991,10
2222949,1362738,1151991,1151991,11519911151991,10
2506437,1533818,1151991,1151991,11519911151991,10
2844522,1738007,1151991,1151991,11519911151991,10


In [ ]:
train.loc[train.pair_target == 1745100174510,['domain','target','pair_target','pos']]

,domain,target,pair_target,pos


In [ ]:
train.loc[train['event_group_id']==559390,['domain','target','pair_target','pos']]

,domain,target,pair_target,pos
910986,574621,1151991,11519910574621,0
910987,95524,1151991,11519910095524,1
910988,574675,1151991,11519910574675,2
910989,733020,1151991,11519910733020,3
910990,393095,1151991,11519910393095,4
910991,443041,1151991,11519910443041,5
910992,574621,1151991,11519910574621,6
910993,209463,1151991,11519910209463,7
910994,944858,1151991,11519910944858,8
910995,39350,1151991,11519910039350,9


In [ ]:
train.loc[train.target == 174510,['event_group_id','domain','target','pair_target','pos']]

,event_group_id,domain,target,pair_target,pos
939818,567162,574746,174510,1745100574746,0
939819,567162,1217173,174510,1745101217173,1
939820,567162,417740,174510,1745100417740,2
939821,567162,669660,174510,1745100669660,3
939822,567162,574746,174510,1745100574746,4
939823,567162,515306,174510,1745100515306,5
939824,567162,865446,174510,1745100865446,6
939825,567162,167665,174510,1745100167665,7
939826,567162,647587,174510,1745100647587,8
939827,567162,834629,174510,1745100834629,9


In [ ]:
train.loc[train.pair_target == 11519911151991,['event_group_id','domain','target','pair_target','pos']]

,event_group_id,domain,target,pair_target,pos
927101,559390,1151991,1151991,11519911151991,10
1069958,645559,1151991,1151991,11519911151991,10
1474461,890785,1151991,1151991,11519911151991,10
1985279,1197961,1151991,1151991,11519911151991,10
2261951,1362738,1151991,1151991,11519911151991,10
2550415,1533818,1151991,1151991,11519911151991,10
2894385,1738007,1151991,1151991,11519911151991,10


In [ ]:
train.columns

Index(['event_group_id', 'device_id', 'timestamp', 'domain', 'url', 'pos', 'y',
       'ff_count_slash', 'ff_count_subdomain', 'target', 'url_target',
       'ff_count_slash_target', 'ff_count_subdomain_target', 'pair_target',
       'ff_pair_count_domain', 'ff_pair_count_root_domain',
       'ff_pair_count_ltd_domain', 'ff_pair_count_cat_domain',
       'ff_pair_count_url', 'ff_pair_count_path_url', 'ff_count_dev_ref',
       'ff_count_dom_ref', 'ff_count_dev_root_ref', 'ff_count_dom_root_ref',
       'ff_count_dev_ltd_ref', 'ff_count_dom_ltd_ref', 'ff_count_dev_cat_ref',
       'ff_count_dom_cat_ref', 'ff_count_dev_url_ref', 'ff_count_dom_url_ref',
       'ff_count_dev_path_url_ref', 'ff_count_dom_path_url_ref',
       'ff_count_root_ref_root_domain', 'ff_count_ltd_ref_ltd_domain',
       'ff_count_cat_ref_cat_domain', 'ff_count_url_ref_url',
       'ff_count_path_url_ref_path_url', 'ff_prob_ref', 'ff_prob_ref_root',
       'ff_prob_ref_ltd', 'ff_prob_ref_cat', 'ff_prob_ref_url',
   